<a href="https://colab.research.google.com/github/cristiancs/TMMC-PRACTICA-1/blob/master/ALTO_RENDIMIENTO_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2019-09-26 19:00:27--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.189.146
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.189.146|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?9NIpp4aiM0-iJOBntOKMm0dhDV80YY1LwCrhf7ZoXMD3nm8A3KK9BIKdkuv6LFnTKL9o3O9cwoWm9WZQVKlPgpBAuyO-vXk_37tqjJ-DmobrIAo77aDzyO-Ms6CniQiED2O13QRRMoAAnVeCXsewO-H7mG5zfFIBuLLcIiSY9UB0CPhqA54j3t6fnVdPzutnPsvB9WbV_RZwQMdHWUk [following]
--2019-09-26 19:00:28--  https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?9NIpp4aiM0-iJOBntOKMm0dhDV80YY1LwCrhf7ZoXMD3nm8A3KK9BIKdkuv6LFnTKL9o3O9cwoWm9WZQVKlPgpBAuyO-vXk_37tqjJ-DmobrIAo77aDzyO

In [0]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

%load_ext nvcc_plugin


  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-gz8bwse_
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-gz8bwse_
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=7789a37ad5c97a9c6aadbb4b52662d8a13e2ac6a9598c65723aa63dcdd5b87b3
  Stored in directory: /tmp/pip-ephem-wheel-cache-kjxt6l39/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [0]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [0]:
from google.colab import files
uploaded = files.upload()



Saving data.txt.zip to data.txt.zip


In [0]:
!unzip "data.txt.zip"

Archive:  data.txt.zip
  inflating: data.txt                
   creating: __MACOSX/
  inflating: __MACOSX/._data.txt     


In [0]:
%%cu
#include <iostream>
#include <fstream>
#include <string>
#include <stdio.h>
#include <cstring>

#include <stdlib.h>
using namespace std;


int calcular_materia(int* data, int M, int N) {
    int suma = 0;
    for (int i = 0; i < M*N; ++i) {
	    suma+= data[i*4];
        suma+= data[i*4+1];
        suma+= data[i*4+2];
        suma+= data[i*4+3];
	}
   return suma;
}

__device__ int atomicSet(int* address, int val){
    int old = *address, assumed;
    do{
        assumed = old;
        old = atomicCAS(address, assumed, val);
    } while(assumed != old);
    return old;
}
__global__ void colision(int* data,  int* salida, int M, int N) {
	int tId = threadIdx.x + blockIdx.x * blockDim.x;

	
	if(tId < N * M) {
        int top, bottom, left, right;
        right = data[tId*4+0];
        top   = data[tId*4+1];
        left  = data[tId*4+2];
        bottom  = data[tId*4+3];
        // Hay Colision
        if((right+top+left+bottom) == 2 ) {
            if((top + bottom) == 2) {

                left = 1;
                right = 1;
                top = 0;
                bottom = 0;
            } else if((left + right) == 2) {
                left = 0;
                right = 0;
                top = 1;
                bottom = 1;
            }
        }
        salida[tId*4+0] = right;
        salida[tId*4+1] = top;
        salida[tId*4+2] = left;
        salida[tId*4+3] = bottom;
	}
}


__global__ void stream(int* data,  int* salida, int M, int N) {
	int tId = threadIdx.x + blockIdx.x * blockDim.x;

	
	if(tId < N * M) {
        int top, bottom, left, right, x, y;
        right = data[tId*4+0];
        top   = data[tId*4+1];
        left  = data[tId*4+2];
        bottom  = data[tId*4+3];

        x = tId%M;
        y = tId/M;
        
        int tId_next_top = y-1 == -1 ? N*M - (M-x) : tId - M ;
        int tId_next_bottom = y+1 == N ? x : tId + M;
        int tId_next_left = x-1 == -1 ? tId + M - 1 : tId - 1 ;
        int tId_next_right = x+1 == M ? tId - M + 1 : tId + 1 ;

        if(right == 1) {
            atomicSub(&salida[tId*4+0], 1);
            atomicAdd(&salida[tId_next_right*4+0], 1);
        }
        if(top == 1) {
            atomicSub(&salida[tId*4+1], 1);
            atomicAdd(&salida[tId_next_top*4+1], 1);
        }
        if(left == 1) {
            // El problema es que esta hebra hace 0
            // pero la siguiente hebra si quiere mover a esta posición debe dejarla en 1
            // por tanto dependiendo de quien trabaje primero
            // va a quedar con 0 o 1
            atomicSub(&salida[tId*4+2], 1);
            atomicAdd(&salida[tId_next_left*4+2], 1);
        }
        if(bottom == 1) {
            atomicSub(&salida[tId*4+3], 1);
            atomicAdd(&salida[tId_next_bottom*4+3], 1);
        }
	}
}


int *CalcularAoS(string filename, int *M, int *N)
{
    char cstr[filename.size() + 1];
    strcpy(cstr, filename.c_str());
    FILE *in = fopen(cstr, "r");
    fscanf(in, "%d %d", M, N);

    int *AoS = new int[4 * *M * *N];

    for (int i = 0; i < 4; i++)
    {
        for (int j = 0; j < *M * *N; j++)
        {
            fscanf(in, "%d", &AoS[i + (j * 4)]);
        }
    }
    
    fclose(in);
    return AoS;
}

int main(int argc, char const *argv[])
{

    int M, N;
    int *AoS = CalcularAoS("data.txt", &M, &N);
 
    cudaEvent_t ct1, ct2;
    int block_size = 256;
    int grid_size = (int) ceil( (float) M * N / block_size);
 
    int* data_gpu, *salida_gpu;
    float dt;
    cudaMalloc(&data_gpu, sizeof(int) * N * M * 4);
    cudaMalloc(&salida_gpu, sizeof(int) * N * M * 4);
    cudaMemcpy(data_gpu, AoS, sizeof(int) * N * M * 4, cudaMemcpyHostToDevice);

    cout << calcular_materia(AoS, M, N) << endl;
    cudaEventCreate(&ct1);
	cudaEventCreate(&ct2);
	cudaEventRecord(ct1);

    for(int i = 0; i < 100; i++) {
        colision<<<grid_size, block_size>>>(data_gpu, salida_gpu, M, N);
        cudaDeviceSynchronize();
        stream<<<grid_size, block_size>>>(salida_gpu, data_gpu, M, N);
        cudaDeviceSynchronize();
        cudaMemcpy(AoS, data_gpu, sizeof(int) * N * M * 4, cudaMemcpyDeviceToHost);
        cout << "a stream: "  << calcular_materia(AoS, M, N) << endl;
    }
	

	cudaEventRecord(ct2);
	cudaEventSynchronize(ct2);
	cudaEventElapsedTime(&dt, ct1, ct2);

    cout << "Tiempo: " << dt << "[ms]" << '\n';
 
    
    return 0;
}

5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a stream: 5999701
a 

In [0]:
%%cu
#include <iostream>
#include <fstream>
#include <string>
#include <stdio.h>
#include <cstring>

#include <stdlib.h>
using namespace std;


int calcular_materia(int* data, int M, int N) {
    int suma = 0;
    for (int i = 0; i < M*N; ++i) {
	    suma+= data[i];
      suma+= data[i+(M*N*1)];
      suma+= data[i+(M*N*2)];
      suma+= data[i+(M*N*3)];
	}
   return suma;
}

__device__ int atomicSet(int* address, int val){
    int old = *address, assumed;
    do{
        assumed = old;
        old = atomicCAS(address, assumed, val);
    } while(assumed != old);
    return old;
}
__global__ void colision(int* data,  int* salida, int M, int N) {
	int tId = threadIdx.x + blockIdx.x * blockDim.x;

	
	if(tId < N * M) {
        int top, bottom, left, right;
        right = data[tId+(M*N*0)];
        top   = data[tId+(M*N*1)];
        left  = data[tId+(M*N*2)];
        bottom  = data[tId+(M*N*3)];
        // Hay Colision
        if((right+top+left+bottom) == 2 ) {
            if((top + bottom) == 2) {

                left = 1;
                right = 1;
                top = 0;
                bottom = 0;
            } else if((left + right) == 2) {
                left = 0;
                right = 0;
                top = 1;
                bottom = 1;
            }
        }
        salida[tId+(M*N*0)] = right;
        salida[tId+(M*N*1)] = top;
        salida[tId+(M*N*2)] = left;
        salida[tId+(M*N*3)] = bottom;
	}
}


__global__ void stream(int* data,  int* salida, int M, int N) {
	int tId = threadIdx.x + blockIdx.x * blockDim.x;

	if(tId < N * M) {
        int top, bottom, left, right, x, y;
        right = data[tId+(M*N*0)];
        top   = data[tId+(M*N*1)];
        left  = data[tId+(M*N*2)];
        bottom  = data[tId+(M*N*3)];

        x = tId%M;
        y = tId/M;
        
        int tId_next_top = y-1 == -1 ? N*M - (M-x) : tId - M ;
        int tId_next_bottom = y+1 == N ? x : tId + M;
        int tId_next_left = x-1 == -1 ? tId + M - 1 : tId - 1 ;
        int tId_next_right = x+1 == M ? tId - M + 1 : tId + 1 ;

        if(right == 1) {
            atomicSub(&salida[tId+(M*N*0)], 1);
            atomicAdd(&salida[tId_next_right+(M*N*0)], 1);
        }
        if(top == 1) {
            atomicSub(&salida[tId+(M*N*1)], 1);
            atomicAdd(&salida[tId_next_top+(M*N*1)], 1);
        }
        if(left == 1) {
            // El problema es que esta hebra hace 0
            // pero la siguiente hebra si quiere mover a esta posición debe dejarla en 1
            // por tanto dependiendo de quien trabaje primero
            // va a quedar con 0 o 1
            atomicSub(&salida[tId+(M*N*2)], 1);
            atomicAdd(&salida[tId_next_left+(M*N*2)], 1);
        }
        if(bottom == 1) {
            atomicSub(&salida[tId+(M*N*3)], 1);
            atomicAdd(&salida[tId_next_bottom+(M*N*3)], 1);
        }
	}
}

int* CalcularSoA(string filename, int *N, int *M) {
    char cstr[filename.size() + 1];
    strcpy(cstr, filename.c_str());
    FILE *in = fopen(cstr, "r");
    fscanf(in, "%d %d", M, N);

    int *SoA = new int[4 * *M * *N];
    for (int i = 0; i < 4; i++)
    {
        for (int j = 0; j < *M * *N; ++j)
        {
            fscanf(in, "%d", &SoA[(i * *M * *N) + j]);
        }
    }
    fclose(in);
    return SoA;
} 

int main(int argc, char const *argv[])
{

    int M, N;
    int *SoA = CalcularSoA("data.txt", &M, &N);
 
    cudaEvent_t ct1, ct2;
    int block_size = 256;
    int grid_size = (int) ceil( (float) M * N / block_size);
 
    int* data_gpu, *salida_gpu;
    float dt;
    cudaMalloc(&data_gpu, sizeof(int) * N * M * 4);
    cudaMalloc(&salida_gpu, sizeof(int) * N * M * 4);
    cudaMemcpy(data_gpu, SoA, sizeof(int) * N * M * 4, cudaMemcpyHostToDevice);

    cout << calcular_materia(SoA, M, N) << endl;
    cudaEventCreate(&ct1);
	cudaEventCreate(&ct2);
	cudaEventRecord(ct1);

    for(int i = 0; i < 100; i++) {
        colision<<<grid_size, block_size>>>(data_gpu, salida_gpu, M, N);
        cudaDeviceSynchronize();
        stream<<<grid_size, block_size>>>(salida_gpu, data_gpu, M, N);
        cudaDeviceSynchronize();
        cudaMemcpy(SoA, data_gpu, sizeof(int) * N * M * 4, cudaMemcpyDeviceToHost);
        cout << "a stream: "  << calcular_materia(SoA, M, N) << endl;
    }
	

	cudaEventRecord(ct2);
	cudaEventSynchronize(ct2);
	cudaEventElapsedTime(&dt, ct1, ct2);

    cout << "Tiempo: " << dt << "[ms]" << '\n';
 
    
    return 0;
}

In [0]:
!ls

sample_data  src


In [0]:
%%cu
#include <iostream>
#include <fstream>
#include <string>
#include <stdio.h>
#include <cstring>

#include <stdlib.h>
using namespace std;

int calcular_materia(int* data, int M, int N) {
    int suma = 0;
    for (int i = 0; i < M*N; ++i) {
	    suma+= data[i*4];
        suma+= data[i*4+1];
        suma+= data[i*4+2];
        suma+= data[i*4+3];
	}
   return suma;
}

__device__ int atomicSet(int* address, int val){
    int old = *address, assumed;
    do{
        assumed = old;
        old = atomicCAS(address, assumed, val);
    } while(assumed != old);
    return old;
}
__global__ void colision(int* data,  int* salida, int M, int N) {
	int tId = threadIdx.x + blockIdx.x * blockDim.x;

	
	if(tId < N * M) {
        int top, bottom, left, right;
        right = data[tId*4+0];
        top   = data[tId*4+1];
        left  = data[tId*4+2];
        bottom  = data[tId*4+3];
        // Hay Colision
        if((right+top+left+bottom) == 2 ) {
            if((top + bottom) == 2) {

                left = 1;
                right = 1;
                top = 0;
                bottom = 0;
            } else if((left + right) == 2) {
                left = 0;
                right = 0;
                top = 1;
                bottom = 1;
            }
        }
        salida[tId*4+0] = right;
        salida[tId*4+1] = top;
        salida[tId*4+2] = left;
        salida[tId*4+3] = bottom;
	}
}


__global__ void stream(int* data,  int* salida, int M, int N) {
	int tId = threadIdx.x + blockIdx.x * blockDim.x;

	
	if(tId < N * M) {
        int top, bottom, left, right, x, y;
        right = data[tId*4+0];
        top   = data[tId*4+1];
        left  = data[tId*4+2];
        bottom  = data[tId*4+3];

        x = tId%M;
        y = tId/M;
        
        int tId_next_top = y-1 == 0 ? N*M - (M-x) : tId - M ;
        int tId_next_bottom = y+1 == N ? x : tId + M;
        int tId_next_left = x-1 == -1 ? tId + M - 1 : tId - 1 ;
        int tId_next_right = x+1 == M ? tId - M + 1 : tId + 1 ;

        if(right == 1) {
            atomicSub(&salida[tId*4+0], 1);
            atomicAdd(&salida[tId_next_left*4+0], 1);
        }
        if(top == 1) {
            atomicSub(&salida[tId*4+1], 1);
            atomicAdd(&salida[tId_next_left*4+1], 1);
        }
        if(left == 1) {
            // El problema es que esta hebra hace 0
            // pero la siguiente hebra si quiere mover a esta posición debe dejarla en 1
            // por tanto dependiendo de quien trabaje primero
            // va a quedar con 0 o 1
            atomicSub(&salida[tId*4+2], 1);
            atomicAdd(&salida[tId_next_left*4+2], 1);
        }
        if(bottom == 1) {
            atomicSub(&salida[tId*4+3], 1);
            atomicAdd(&salida[tId_next_left*4+3], 1);
        }
	}
}


int *CalcularAoS(string filename, int *M, int *N)
{
    char cstr[filename.size() + 1];
    strcpy(cstr, filename.c_str());
    FILE *in = fopen(cstr, "r");
    fscanf(in, "%d %d", M, N);

    int *AoS = new int[4 * *M * *N];

    for (int i = 0; i < 4; i++)
    {
        for (int j = 0; j < *M * *N; j++)
        {
            fscanf(in, "%d", &AoS[i + (j * 4)]);
        }
    }
    
    fclose(in);
    return AoS;
}

int* CalcularSoA(string filename, int *N, int *M) {
    int* particulas = new int[4 * N * M];

    for (int i = 0; i < 4; i++)
    {
        for (int j = 0; j < M * N; ++j)
        {
            fscanf(in, "%d", &particulas[(i * M * N) + j]);
        }
    }

    return particulas;
} 

int main(int argc, char const *argv[])
{

    int M, N;
    int *SoA = CalcularSoA("data.txt", &M, &N);
 
    cudaEvent_t ct1, ct2;
    int block_size = 256;
    int grid_size = (int) ceil( (float) M * N / block_size);
 
    int* data_gpu, *salida_gpu;
    float dt;
    cudaMalloc(&data_gpu, sizeof(int) * N * M * 4);
    cudaMalloc(&salida_gpu, sizeof(int) * N * M * 4);
    cudaMemcpy(data_gpu, AoS, sizeof(int) * N * M * 4, cudaMemcpyHostToDevice);

    cout << calcular_materia(AoS, M, N) << endl;
    cudaEventCreate(&ct1);
	cudaEventCreate(&ct2);
	cudaEventRecord(ct1);

    for(int i = 0; i < 100; i++) {
        colision<<<grid_size, block_size>>>(data_gpu, salida_gpu, M, N);
        cudaDeviceSynchronize();
        stream<<<grid_size, block_size>>>(salida_gpu, data_gpu, M, N);
        cudaDeviceSynchronize();
        cudaMemcpy(AoS, data_gpu, sizeof(int) * N * M * 4, cudaMemcpyDeviceToHost);
        cout << "a stream: "  << calcular_materia(AoS, M, N) << endl;
    }
	

	cudaEventRecord(ct2);
	cudaEventSynchronize(ct2);
	cudaEventElapsedTime(&dt, ct1, ct2);

    cout << "Tiempo: " << dt << "[ms]" << '\n';
 
    
    return 0;
}

# Parte B

In [0]:
m%%cu
#include <iostream>
#include <fstream>
#include <string>
#include <stdio.h>
#include <cstring>

#include <stdlib.h>
using namespace std;


int calcular_materia(int* data, int M, int N) {
    int suma = 0, top, bottom, left, right;
    for (int i = 0; i < M*N; ++i) {
        right = (data[i] & ( 1 << 0)) >> 0;
        top   = (data[i] & ( 1 << 1 )) >> 1;
        left  = (data[i] & ( 1 << 2 )) >> 2;
        bottom  = (data[i] & ( 1 << 3)) >> 3;
        
        suma+=right+top+left+bottom;
        
	}
   return suma;
}
__device__ int atomicSet(int* address, int val){
    int old = *address, assumed;
    do{
        assumed = old;
        old = atomicCAS(address, assumed, val);
    } while(assumed != old);
    return old;
}

__global__  void colision(int* data,  int* salida, int M, int N) {
	int tId = threadIdx.x + blockIdx.x * blockDim.x;

	
	if(tId < N * M) {
        int top, bottom, left, right, toSave=data[tId];
        right = (toSave & ( 1 << 0)) >> 0;
        top   = (toSave & ( 1 << 1 )) >> 1;
        left  = (toSave & ( 1 << 2 )) >> 2;
        bottom  = (toSave & ( 1 << 3)) >> 3;

        // Hay Colision
        if((right+top+left+bottom) == 2 ) {
            if((top + bottom) == 2) {
                toSave = 0b0101;
            } else if((left + right) == 2) {
                toSave = 0b1010;
            }
        }
        salida[tId] = toSave;
	}
}


__global__ void stream(int* data,  int* salida, int M, int N) {
	int tId = threadIdx.x + blockIdx.x * blockDim.x;

	
	if(tId < N * M) {
        int top_vecino, bottom_vecino, left_vecino, right_vecino, x, y;


        right = (lectura & 0b1) > 0;
        top   = (lectura & 0b10) > 0;
        left  = (lectura & 0b100) > 0;
        bottom  = (lectura & 0b1000) > 0;

        x = tId%M;
        y = tId/M;
        
        int tId_next_top = y-1 == 0 ? N*M - (M-x) : tId - M ;
        int tId_next_bottom = y+1 == N ? x : tId + M;
        int tId_next_left = x-1 == -1 ? tId + M - 1 : tId - 1 ;
        int tId_next_right = x+1 == M ? tId - M + 1 : tId + 1 ;

        right_vecino = data[tId_next_left];
        if(right_vecino == 1) {
            atomicSet(&salida[tId], data[tId]-1);
        }
        if(top == 1) {
            atomicSub(&salida[tId], 2);
            atomicAdd(&salida[tId_next_top], 2);
        }
        if(left == 1) {
            // El problema es que esta hebra hace 0
            // pero la siguiente hebra si quiere mover a esta posición debe dejarla en 1
            // por tanto dependiendo de quien trabaje primero
            // va a quedar con 0 o 1
            atomicSub(&salida[tId], 4);
            atomicAdd(&salida[tId_next_left], 4);
        }
        if(bottom == 1) {
            atomicSub(&salida[tId], 8);
            atomicAdd(&salida[tId_next_bottom], 8);
        }
	}
}


int *Read2Binary(string filename, int *M, int *N)
{
    char cstr[filename.size() + 1];
    strcpy(cstr, filename.c_str());
    FILE *in = fopen(cstr, "r");
    fscanf(in, "%d %d", M, N);
    int *data = new int[*M * *N];
    int suma = 0;
    for (int i = 0; i < 4; i++)
    {
        int lectura;
        for (int j = 0; j < *M * *N; j++)
        {
            fscanf(in, "%d", &lectura );
            if(lectura == 1) {
                suma+=1;
            }
            lectura = lectura << i;
           
            data[j] =  data[j] | lectura;
        }
    }
    cout << "Iones segun lectura: " << suma << endl;
    fclose(in);
    return data;
}

int main(int argc, char const *argv[])
{

    int M, N;
    int *data = Read2Binary("data.txt", &M, &N);
 
    cudaEvent_t ct1, ct2;
    int block_size = 256;
    int grid_size = (int) ceil( (float) M * N / block_size);
 
    int* data_gpu, *salida_gpu;
    float dt;
    cudaMalloc(&data_gpu, sizeof(int) * N * M );
    cudaMalloc(&salida_gpu, sizeof(int) * N * M );
    cudaMemcpy(data_gpu, data, sizeof(int) * N * M, cudaMemcpyHostToDevice);

    cout << calcular_materia(data, M, N) << endl;
    cudaEventCreate(&ct1);
	cudaEventCreate(&ct2);
	cudaEventRecord(ct1);

    for(int i = 0; i < 100; i++) {
        colision<<<grid_size, block_size>>>(data_gpu, salida_gpu, M, N);
        cudaDeviceSynchronize();
        stream<<<grid_size, block_size>>>(salida_gpu, data_gpu, M, N);
        cudaDeviceSynchronize();
        cudaMemcpy(data, data_gpu, sizeof(int) * N * M, cudaMemcpyDeviceToHost);
        cout << "a stream: "  << calcular_materia(data, M, N) << endl;
    }
	

	cudaEventRecord(ct2);
	cudaEventSynchronize(ct2);
	cudaEventElapsedTime(&dt, ct1, ct2);

    cout << "Tiempo: " << dt << "[ms]" << '\n';
 
    
    return 0;
}

Iones segun lectura: 5999701
5999701
a stream: 5998845
a stream: 5998012
a stream: 5997098
a stream: 5996477
a stream: 5997189
a stream: 5998415
a stream: 5998960
a stream: 5998445
a stream: 5998005
a stream: 6000800
a stream: 5999394
a stream: 5997222
a stream: 5997631
a stream: 5998377
a stream: 5997980
a stream: 5998858
a stream: 5997953
a stream: 5997805
a stream: 5998746
a stream: 5997911
a stream: 5996732
a stream: 5996467
a stream: 5997624
a stream: 5996921
a stream: 5995412
a stream: 5997601
a stream: 5997581
a stream: 5997362
a stream: 5997457
a stream: 5997077
a stream: 5996087
a stream: 5997391
a stream: 5995815
a stream: 5995192
a stream: 5996043
a stream: 5997369
a stream: 5997216
a stream: 5999857
a stream: 5999053
a stream: 5999669
a stream: 5999387
a stream: 5997445
a stream: 5995851
a stream: 5996365
a stream: 5996206
a stream: 5997313
a stream: 5995457
a stream: 5993944
a stream: 5994263
a stream: 5993223
a stream: 5992864
a stream: 5991253
a stream: 5991233
a stream: